In [1]:

var numEdges = 200;
var numVertices = 50;


In [2]:
    var slices =  2;
    var conf = new SparkConf().setAppName("JavaScript transitive closure");
    var sc = new SparkContext(conf);

var slices = slices ? 0+slices: 2;
var tc = sc.parallelizePairs(generateGraph(), slices).cache();


 Linear transitive closure: each round grows paths by one edge,
 by joining the graph's edges with the already-discovered paths.
 e.g. join the path (y, z) from the TC with the edge (x, y) from
 the graph to obtain the path (x, z).


 Because join() joins on keys, the edges are stored in reversed order.


In [3]:
var edges = tc.mapToPair(function(tuple) {
    return new Tuple(tuple[1], tuple[0]);
});


In [4]:
var oldCount;
var nextCount = tc.count();
do {
    oldCount = nextCount;
    // Perform the join, obtaining an RDD of (y, (z, x)) pairs,
    // then project the result to obtain the new (x, z) paths.
    tc = tc.union(tc.join(edges).mapToPair(function(triple){
        return new Tuple(triple[1][1],triple[1][0]);
    })).distinct().cache();
    nextCount = tc.count();

} while (nextCount != oldCount);

var result = tc.count();
    print("TC has " + result + " edges.");

    sc.stop();
